<a href="https://colab.research.google.com/github/nildodnjunior/rna/blob/master/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-mnist > /dev/null
!pip install wget > /dev/null
!pip install sh > /dev/null

In [2]:
import wget
import os
from sh import gunzip
from mnist import MNIST
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin

Função que baixa os arquivos do dataset do site http://yann.lecun.com/exdb/mnist/index.html e retorna as imagens e labels de treino e teste.

In [3]:
def load_mnist():
    if not os.path.exists('./mnist'):
        os.makedirs("./mnist")

    if 'train-images-idx3-ubyte' not in os.listdir('mnist'):
        wget.download('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', 'mnist')
        gunzip('mnist/train-images-idx3-ubyte.gz')
    if 'train-labels-idx1-ubyte' not in os.listdir('mnist'):
        wget.download('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', 'mnist')
        gunzip('mnist/train-labels-idx1-ubyte.gz')
    if 't10k-images-idx3-ubyte' not in os.listdir('mnist'):
        wget.download('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', 'mnist')
        gunzip('mnist/t10k-images-idx3-ubyte.gz')
    if 't10k-labels-idx1-ubyte' not in os.listdir('mnist'):
        wget.download('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', 'mnist')
        gunzip('mnist/t10k-labels-idx1-ubyte.gz')

    mnist = MNIST('mnist')
    train_images, train_labels = mnist.load_training()
    test_images, test_labels = mnist.load_testing()

    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    return (train_images, train_labels), (test_images, test_labels)

In [4]:
def define_arquitetura(input_shape, output_shape):
  inputs = keras.Input(shape=input_shape)
  x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
  x = layers.MaxPooling2D(pool_size=2)(x)
  x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
  x = layers.MaxPooling2D(pool_size=2)(x)
  x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
  x = layers.Flatten()(x)
  outputs = layers.Dense(output_shape, activation="softmax")(x)
  return keras.Model(inputs=inputs, outputs=outputs)

class DividePor255(BaseEstimator, TransformerMixin):
  def fit(self, X, y):
    return self
  def transform(self, X, y=None):
    return X.astype("float32") / 255

class MudaShape(BaseEstimator, TransformerMixin):
  def fit(self, X, y):
    return self
  def transform(self, X, y=None):
    return X.reshape((-1, 28, 28, 1))

In [5]:
class RedeNeural(BaseEstimator, ClassifierMixin):
  def __init__(self, epochs=5, batch_size=128):
    self.epochs = epochs
    self.batch_size = batch_size
  def fit(self, X, y):
    self.labels, ids = np.unique(y, return_inverse=True)
    yhot = keras.utils.to_categorical(ids)
    self.model = define_arquitetura(X.shape[1:], yhot.shape[1])
    self.model.compile(optimizer="rmsprop",
                       loss="categorical_crossentropy",
                       metrics=["accuracy"])
    self.model.fit(X, yhot, epochs=self.epochs, batch_size=self.batch_size)
    return self
  def predict(self, X, y=None):
    probabilities = self.model.predict(X)
    ypred = self.labels[np.argmax(probabilities, axis=1)]
    return ypred

In [6]:
(train_images, train_labels), (test_images, test_labels) = load_mnist()

/usr/local/lib/python3.10/dist-packages/sh.py:1980: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [7]:
modelo = Pipeline([
    ("scaler", DividePor255()),
    ("reshape", MudaShape()),
    ("ann", RedeNeural())
])

modelo.fit(train_images, train_labels)
ypred = modelo.predict(test_images)
accuracy_score(test_labels, ypred)

Epoch 1/5
469/469 [==============================] - 12s 9ms/step - loss: 0.2246 - accuracy: 0.9309
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0524 - accuracy: 0.9837
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0352 - accuracy: 0.9891
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0258 - accuracy: 0.9919
Epoch 5/5
313/313 [==============================] - 1s 2ms/step


0.9915